In [1]:

# 本帖最后由 xiaohui 于 2018-11-11 15:12 编辑

# 执行下面这些代码条件：
# 需要部署提供运行这些代码的环境
# 安装代码中的外部模块
# 代码中的有些参数需要改成实际的参数，比如Cookie,url,这些
# 分析代码中的路径，在相应路径下建立对应的目录


# 以下就是源代码

#_*_ coding:utf-8 _*_
#导入若干模块
import os
import re
import sys
import requests
import time
from Crypto.Cipher import AES
import subprocess
from binascii import b2a_hex, a2b_hex
#-----------------------------------------------------------------------
#公共请求头/全局通用
headers={
        'Accept': '* / *',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Host':'videotts.it211.com.cn',
        'Origin':'http://tts.tmooc.cn',
        'Referer':'http://tts.tmooc.cn/video/showVideo?menuId=691210&version=AIDTN201903',
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
        'Connection': 'close',
        'Cookie':'54D0E601D8784E7CAAC8E9A9616BB2D2|E_bfuo1l9'
    }
#-----------------------------------------------------------------------
#获取m3u8url
def get_ts_url(url,filenames):
    #请求m3u8集合文件
    try:
        m3u8=requests.get(url, headers=headers,timeout=10).text
        #提取文件中的秘钥key地址
        key_url=re.findall(r'EXT-X-KEY:METHOD=AES-128,URI="(.*)"', m3u8)[0]
    except:
        return 10
    #判断是否是正常的m3u8文件
    if "EXTINF" in m3u8:
        #获取m3u８中的每一行
        file_line = m3u8.split('\n')
        #循环取出每个ts文件的url,并且生成列表
        for line in file_line:
            if '.ts' in line:
                #生成文件名称
                file_name=line.split('-')[1]
                #循环下载每个ts文件
                error1=get_ts(line,file_name,key_url)
                if error1==10:
                    return error1
        # ts全部下载完成后开始合并
        hebin_videos(filenames)
    else:
        print('此文件不是m3u8格式的标准文件！！')
        #退出程序
        exit()
#----------------------------------------------------------------------------
#下载每个ｔｓ文件
def get_ts(url,file_name,key_url):
    #获取ｔｓ二进制文件
    video=requests.get(url,headers=headers,timeout=10).content
    #判断文件是否为空
    if len(video):
        #获取秘钥ｋｅｙ
        key=get_key(key_url)
        #使用aes开始解密算法
        cryptor = AES.new(key, AES.MODE_CBC, key)
        #开始保存ts文件到本地
        with open(os.getcwd() + '/videos/'+file_name, 'ab') as f:
            #解密并且写入保存
            try:
                f.write(cryptor.decrypt(video))
            except:
                return 10

        print(file_name+"下载成功")
    else:
        print('视频获取失败！！')
        exit()
#----------------------------------------------------------------------------
# 获取秘钥文件
def get_key(key_url):
    #秘钥文件地址
    url=key_url
    #请求秘钥
    key=requests.get(url, headers=headers,timeout=10).content
    return key
    pass
#----------------------------------------------------------------------------
#排序处理
def sorts(videos_path):
    file = []
    # 获取目录下的所有ｔｓ文件
    files = os.listdir(videos_path)
    # 只取出文件数字前zui,然后进行从小到大排序
    for i in files:
        num_file = int(i.split('.')[0])
        # 将文件前zui放进列表
        file.append(num_file)
    # 正序排列
    file.sort()
    return file
#-----------------------------------------------------------------------------
#视频合并
def hebin_videos(filenames):
    file=sorts(os.getcwd() +'/videos')
    #定义存储字符串容器
    long_file=''
    for i in file:
        #字符串拼接处理
        long_file += str(i) + '.ts|'
    #删除最后一位处理
    ts = long_file[:-1]
    #合成整条命令
    os.chdir(os.getcwd()+'/videos')
    cmd='ffmpeg -i "concat:%s" -c copy -bsf:a aac_adtstoasc -movflags +faststart ../one_videos/%s.mp4' % (ts,filenames)
    reqult=subprocess.call(cmd,shell=True)
    if reqult==0:
        print(str(filenames)+'.mp4下载成功')
        #删除所有ｔｓ
        tts=subprocess.call("rm -rf %s" % ('/root/PycharmProjects/untitled1/videos/*'),shell=True)
        if tts!=0:
            print('删除ts文件失败！！,程序退出')
            exit()
    else:
        print('视频合并失败,程序退出！！')
        exit()
#------------------------------------------------------------------------------
#主函数
def mains():
    errors=[]
    # 删除所有ｔｓ
    subprocess.call("rm -rf %s" % ('/root/PycharmProjects/untitled1/videos/*'), shell=True)
    words='http://videotts.it211.com.cn/aid1904a.*/aid1904a(.*).m3u8'
    with open(os.getcwd() + '/m3u8.txt') as fobj:
        for i in fobj:
            os.chdir('/root/PycharmProjects/untitled1')
            try:
                filenames=re.findall(words, i)[0]
            except:
                print('IndexError')
                with open('/root/PycharmProjects/untitled1/error.txt','a+') as fobj1:
                    fobj1.write(i)
                errors.append(i)
                continue
            error=get_ts_url(i.strip(),filenames)
            if error==10:
                errors.append(filenames)
                print(filenames+'跳过')
                with open('/root/PycharmProjects/untitled1/error.txt','a+') as fobj1:
                    fobj1.write(i)
                # 删除所有ｔｓ
                subprocess.call("rm -rf %s" % ('/root/PycharmProjects/untitled1/videos/*'), shell=True)
                continue
    return errors
#-------------------------------------------------------------------------------
error=mains()
print('*'*50+'报错ｔｓ如下'+'*'*50)
for i in error:
    print(i)


ModuleNotFoundError: No module named 'requests'

In [ ]:

本帖最后由 xiaohui 于 2018-11-11 15:11 编辑

运行代码需求：

安装代码中的所有外部模块
分析代码中的所有路径是否存在
修改代码中的参数，改成实际参数
根据报错信息，进行适当的调试



源代码如下：
#导入各类模块
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import win32con
import win32api
import pyperclip
import requests
import re
import time
#----------------------------------------------------------------
#下载案例和ppt
def dowload(fname,url):
    #每次跳转链接
    driver.get(url)
    time.sleep(3)
    #右击
    ActionChains(driver).context_click().click().perform()
    time.sleep(3)
    #按键A
    win32api.keybd_event(65, 0, 0, 0)
    time.sleep(4)
    win32api.keybd_event(65, 0, win32con.KEYEVENTF_KEYUP, 0)
    time.sleep(3)
    #写入剪切板内容
    pyperclip.copy(fname)
    #粘贴
    win32api.keybd_event(17, 0, 0, 0)
    win32api.keybd_event(86, 0, 0, 0)
    time.sleep(1)
    win32api.keybd_event(86, 0, win32con.KEYEVENTF_KEYUP, 0)
    win32api.keybd_event(17, 0, win32con.KEYEVENTF_KEYUP, 0)
    time.sleep(2)
    #回车保存
    win32api.keybd_event(13, 0, 0, 0)
    time.sleep(1)
    win32api.keybd_event(13, 0, win32con.KEYEVENTF_KEYUP, 0)
    print(fname+'文件保存成功！！')
#-----------------------------------------------------------------
def first(Cookie):
    #提示信息
    title="""
请再运行本程序前，就已经人工登陆成功，并且当前位置在达内课程首页！
预先确定好下载位置，需手动下载一个让系统寻址
第一个tab必须是首页
输入法调成小写

注：窗口需最大化，尽可能关闭其他软件   
"""
    #课程首页地址
    url='http://tts.tmooc.cn/studentCenter/toMyttsPage'
    print(title)
    #设置请求头
    headers={
        'Accept': 'text/html, application/xhtml+xml, image/jxr, */*',
        'Accept-Language': 'zh-Hans-CN, zh-Hans; q=0.5',
        'Connection': 'Keep-Alive',
        'Cookie':Cookie,
        'Host': 'tts.tmooc.cn',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299'
    }
    #发送请求
    content=requests.get(url=url,headers=headers).text
    #返回网页文本数据
    return content


def video_m3u8(content,video,Cookie):
    #设置请求头
    headers={
        'Accept': 'text/html, application/xhtml+xml, image/jxr, */*',
        'Accept-Language': 'zh-Hans-CN, zh-Hans; q=0.5',
        'Connection': 'Keep-Alive',
        'Cookie':Cookie,
        'Host': 'tts.tmooc.cn',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299'
    }
    #视频地址提取
    vv = re.findall(video, content)
    url='http://videotts.it211.com.cn'
    for i in vv:
        a=requests.get(i,headers=headers).text
        link=re.findall(r'<p id="active_(.*).m3u8">',a)
        for ii in link:
            m3u8_url='%s/%s/%s'%(url,ii,ii+'.m3u8')
            with open('m3u8.txt','a+') as fobj:
                fobj.write(m3u8_url+'\n')

def anli(content,alppt,alppt_2):
    #案例匹配规则
    a=re.findall(alppt+'>案例',content)
    #循环所有案例
    time.sleep(10)
    for i in a:
        #anli匹配规则
        c=re.findall(alppt_1,i)
        d=re.findall(alppt_2,i)
        dowload(c[0] + '-' + d[0], i)

def PPT(content,alppt,alppt_2):
    #PPT匹配规则
    aa=re.findall(alppt+'>PPT',content)
    #循环所有PPT
    time.sleep(10)
    for i in aa:
        # ppt匹配规则
        cc = re.findall(alppt_1, i)
        dd = re.findall(alppt_2, i)
        dowload(cc[0]+'-'+dd[0],i)
def timeout():
    #临时停顿
    panduan=input('您准备好了吗?Y/N')
    #确定后继续执行
    if panduan != 'Y':
        time.sleep(5)
        exit()

#http://tts.tmooc.cn/ttsPage/LINU ... Y03/COURSE/ppt.html"
#  active_nsd18020510am.m3u8
#http://videotts.it211.com.cn/nsd18020510am/nsd18020510am.m3u8
#first()
#提取
#next=browser.find_elements_by_xpath("//div[@class='next-video-lty']")
      

#通用匹配规则
alppt='href="(http://tts.tmooc.cn/ttsPage/LINUX/NSDTN201801/.*html)"'
alppt_1='http://tts.tmooc.cn/ttsPage/LINUX/NSDTN201801/(.*)/DAY.*'
alppt_2='http://tts.tmooc.cn/ttsPage/LINUX/NSDTN201801/.*/(DAY[0-9]+)/.*'

#通用cookie
video_cookie='.local.language=zh-CN; cloudAuthorityCookie=0; Hm_lvt_51179c297feac072ee8d3f66a55aa1bd=1536831570,1536853760,1536856995,1536910789; Hm_lpvt_51179c297feac072ee8d3f66a55aa1bd=1536910789; TMOOC-SESSION=C5F550CFD2B248D7BAE440E65FD68889; sessionid=C5F550CFD2B248D7BAE440E65FD68889|ued1804n_pm%40tedu.cn; versionListCookie=UEDTN201805; defaultVersionCookie=UEDTN201805; versionAndNamesListCookie=UEDTN201805N22NUE%25E4%25BA%25A4%25E4%25BA%2592%25E5%25B8%2588%25E5%2585%25A8%25E6%2597%25A5%25E5%2588%25B6%25E8%25AF%25BE%25E7%25A8%258BV15; courseCookie=UED; stuClaIdCookie=583412; isCenterCookie=yes; Hm_lvt_e997f0189b675e95bb22e0f8e2b5fa74=1536441239,1536853910,1536857030,1536910822; JSESSIONID=D2E06266D45A9C02D285BD9954C64259; Hm_lpvt_e997f0189b675e95bb22e0f8e2b5fa74=1536910828'



# 视频匹配规则
video_re = '(http://tts.tmooc.cn/video/showVideo?.*)"'
#获取网页源代码
web_code=first(video_cookie)

#---------------------------------------------------------------------------------------
#视频m3u8获取
video_m3u8(web_code,video_re,video_cookie)

#打开Chrome浏览器
#driver=webdriver.Chrome()

#timeout()

#案例获取
#anli(web_code,alppt,alppt_2)

#timeout()

#ppt获取
#PPT(web_code,alppt,alppt_2)
